In [1]:
# pip install xgboost
# pip install lightgbm
# pip install catboost
import gc
import os
import pandas as pd
import catboost as cbt
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression

In [ ]:
pd.set_option('max_columns', None)
pd.set_option('max_rows', None)

In [2]:
seed = 2022
df_feature = pd.read_pickle('data/feature.pkl')
for f in df_feature.select_dtypes('object').columns:
    lbl = LabelEncoder()
    df_feature[f] = lbl.fit_transform(df_feature[f].astype(str))

df_train = df_feature[df_feature.isDefault.notna()].copy()
df_test = df_feature[df_feature.isDefault.isna()].copy()
df_train.shape, df_test.shape

((800000, 341), (200000, 341))

In [ ]:
# df_train.sample(5)

In [5]:
ycol = 'isDefault'
feature_names = list(
    filter(lambda x: x not in [ycol, 'id'], df_train.columns))

In [34]:
# 逻辑回归
df_train.replace([np.inf, -np.inf], np.nan, inplace=True)
for column in list(df_train.columns[df_train.isnull().sum() > 0]):
    mean_val = df_train[column].mean()
    df_train[column].fillna(mean_val, inplace=True)

df_test.replace([np.inf, -np.inf], np.nan, inplace=True)
for column in list(df_test.columns[df_test.isnull().sum() > 0]):
    mean_val = df_test[column].mean()
    df_test[column].fillna(mean_val, inplace=True)

In [35]:
# dense_features = ['annualIncome', 'delinquency_2years', 'dti', 'employmentLength', 'ficoRangeHigh',
#                   'ficoRangeLow', 'installment', 'interestRate', 'loanAmnt', 'openAcc', 'pubRec',
#                   'pubRecBankruptcies', 'revolBal', 'revolUtil', 'subGrade', 'term', 'totalAcc',
#                   'homeOwnership', 'purpose', 'policyCode']
# df_feature[dense_features] = (df_feature[dense_features] - df_feature[dense_features].mean()) / df_feature[dense_features].std()

df_feature = (df_feature - df_feature.mean()) / df_feature.std()  # 

In [ ]:
df_oof = df_train[['id', ycol]].copy()  # df_train
df_oof['prob'] = 0   # 800000, 2
# train
# id, prob
#  0, x
#  1, x


# 测试集
prediction = df_test[['id']]
prediction['prob'] = 0   # 200000, 2
# test
# id, prob
# 800000, x
# 800001, x


model = LogisticRegression(solver='sag')  # 选逻辑回归作为分类器

# CV5
# 训练集、验证集、测试集
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
for fold_id, (trn_idx, val_idx) in enumerate(kfold.split(df_train[feature_names], df_train[ycol])):
    X_train = df_train.iloc[trn_idx][feature_names]
    Y_train = df_train.iloc[trn_idx][ycol]
    X_val = df_train.iloc[val_idx][feature_names]
    Y_val = df_train.iloc[val_idx][ycol]

    print("\ntraining Fold:{}\n".format(fold_id+1))
    lr_model = model.fit(X_train, Y_train)
    
    pred_val = lr_model.predict_proba(X_val)[:, 1]
    df_oof.loc[val_idx, 'prob'] = pred_val   # 验证集概率

    pred_test = lr_model.predict_proba(df_test[feature_names])[:, 1]
    prediction['prob'] += pred_test / kfold.n_splits
    del lr_model, pred_val, pred_test, X_train, Y_train, X_val, Y_val
    gc.collect()

auc = roc_auc_score(df_oof[ycol], df_oof['prob'])
print('在所有训练集上的auc:', auc)

os.makedirs('sub', exist_ok=True)
sub = prediction[['id', 'prob']].copy()
sub.columns = ['id', 'isDefault']
sub.to_csv('sub/lr_jiedai_{}.csv'.format(auc), index=False)

os.makedirs('prob', exist_ok=True)
prediction.to_csv('prob/lr_test.csv', index=False)
df_oof[['id', 'prob', ycol]].to_csv('prob/lr_train.csv', index=False)

# 连续值归一化: 在所有训练集上的auc: 0.6320645087602705
# 所有值归一化: 在所有训练集上的auc: 0.6320644005936177
# 在所有训练集上的auc: 0.6179237035400197

In [ ]:
X

In [36]:
# 使用xgboost
# pip install xgboost  
# pip install sklearn
# pip install tqdm
# conda install xgboost
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
import warnings
from sklearn.preprocessing import LabelEncoder
import pandas as pd
from sklearn.preprocessing import StandardScaler
import gc
import os
from tqdm import tqdm
import numpy as np

warnings.simplefilter('ignore')


if __name__ == '__main__':   # 代码的起始
    pd.set_option('display.max_columns', None)
    pd.set_option('display.max_rows', None)
    seed = 2022
    df_feature = pd.read_pickle('data/feature.pkl')
    for f in df_feature.select_dtypes('object').columns:
        lbl = LabelEncoder()
        df_feature[f] = lbl.fit_transform(df_feature[f].astype(str))

    

    df_train = df_feature[df_feature.isDefault.notna()].copy()   # 80万
    df_test = df_feature[df_feature.isDefault.isna()].copy()   # 20万

    #df_train = clean_dataset(df_train)
    #df_test = clean_dataset(df_test)
    #df_train = df_train.dropna(axis=0, how='any')   # 删除按nan的行
    df_train.replace([np.inf, -np.inf], np.nan, inplace=True)
    df_train.fillna(df_train.mean(), inplace=True)   # 填充成均值
    df_test.replace([np.inf, -np.inf], np.nan, inplace=True)
    df_test.fillna(df_test.mean(), inplace=True)
    # 特征交叉   性别  年龄   性别年龄
    print(df_train.shape, df_test.shape)   # (800000, 46+xx)

    ycol = 'isDefault'
    feature_names = list(
        filter(lambda x: x not in [ycol, 'id'], df_train.columns))

    # 树不够复杂: 欠拟合   太复杂: 过拟合
    model = xgb.XGBClassifier(max_depth=6,          # 树的最大深度
                              learning_rate=0.05,   # 学习率
                              n_estimators=10000,   # 弱分类器的数目
                              subsample=0.8,        # 构造每棵树 需采样多少样本
                              reg_alpha=10,         # l1/l0正则惩罚系数
                              reg_lambda=12,        # l2正则惩罚系数
                              # tree_method='gpu_hist',
                              random_state=seed)

    # id  prob   # 训练集的预测结果
    df_oof = df_train[['id', ycol]].copy()
    df_oof['prob'] = 0

    # id prob  # 测试的预测结果
    prediction = df_test[['id']]
    prediction['prob'] = 0
    df_importance_list = []

    # 训练集: 用来训练模型
    # 验证集: (来自于训练集) 评估模型的效果
    # 测试集: (没有标签) 需要预测出标签 提交到比赛平台
    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)  # cv5
    for fold_id, (trn_idx, val_idx) in enumerate(
            kfold.split(df_train[feature_names], df_train[ycol])):
        
        X_train = df_train.iloc[trn_idx][feature_names]
        Y_train = df_train.iloc[trn_idx][ycol]
        # print(X_train.shape)   #  800000 / 5 * 4  (640000, 337)  46
        # print(Y_train.shape)   # (640000, 1)   #

        X_val = df_train.iloc[val_idx][feature_names]
        Y_val = df_train.iloc[val_idx][ycol]
        # print(X_val.shape)   # 160000, 337
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_val = scaler.transform(X_val)

        print('\nFold_{} Training ================================\n'.format(
            fold_id + 1))

        # E extreme xgboost  gbdt
        xgb_model = model.fit(X_train,
                              Y_train,
                              eval_set=[(X_train, Y_train), (X_val, Y_val)],
                              verbose=100,
                              eval_metric='auc',
                              early_stopping_rounds=50) # 早停

        pred_val = xgb_model.predict_proba(X_val)[:, 1]   # [0.2, 0.8]
        df_oof.loc[val_idx, 'prob'] = pred_val
        # 训练集   切分  4/5训练  1/5

        #
        pred_test = xgb_model.predict_proba(df_test[feature_names])[:, 1]
        prediction['prob'] += pred_test / kfold.n_splits

        df_importance = pd.DataFrame({
            'column': feature_names,
            'importance': xgb_model.feature_importances_,
        })
        df_importance_list.append(df_importance)

        del xgb_model, pred_val, pred_test, X_train, Y_train, X_val, Y_val
        gc.collect()

    df_importance = pd.concat(df_importance_list)
    df_importance = df_importance.groupby([
        'column'
    ])['importance'].agg('mean').sort_values(ascending=False).reset_index()
    print(df_importance)
    auc = roc_auc_score(df_oof[ycol], df_oof['prob'])
    print('auc:', auc)
    os.makedirs('sub', exist_ok=True)

    sub = prediction[['id', 'prob']].copy()
    sub.columns = ['id', 'isDefault']
    sub.to_csv('sub/jiedai_{}.csv'.format(auc), index=False)
    os.makedirs('prob', exist_ok=True)

    prediction.to_csv('prob/sub_xgb.csv', index=False)
    df_oof[['id', 'prob', ycol]].to_csv('prob/oof_xgb.csv', index=False)

    prediction.to_csv('prob/sub_xgb_{}.csv'.format(auc), index=False)
    df_oof[['id', 'prob', ycol]].to_csv('prob/oof_xgb_{}.csv'.format(auc), index=False)
    # 做缺失值处理: auc: 0.7430078033376142   # CV5  train
    # 不做缺失值处理: auc: 0.7434986214509369

(800000, 341) (200000, 341)

Fold_1 Training ================================

[0]	validation_0-auc:0.70799	validation_1-auc:0.70577
[100]	validation_0-auc:0.74107	validation_1-auc:0.73312
[200]	validation_0-auc:0.75098	validation_1-auc:0.73819
[300]	validation_0-auc:0.75761	validation_1-auc:0.74006
[400]	validation_0-auc:0.76330	validation_1-auc:0.74101
[500]	validation_0-auc:0.76861	validation_1-auc:0.74159
[600]	validation_0-auc:0.77342	validation_1-auc:0.74189
[700]	validation_0-auc:0.77813	validation_1-auc:0.74204
[800]	validation_0-auc:0.78273	validation_1-auc:0.74223
[900]	validation_0-auc:0.78720	validation_1-auc:0.74238
[1000]	validation_0-auc:0.79154	validation_1-auc:0.74251
[1032]	validation_0-auc:0.79288	validation_1-auc:0.74245

Fold_2 Training ================================

[0]	validation_0-auc:0.70821	validation_1-auc:0.70458
[100]	validation_0-auc:0.74134	validation_1-auc:0.73200
[200]	validation_0-auc:0.75120	validation_1-auc:0.73669
[300]	validation_0-auc:0.75798	v

In [37]:
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
import warnings
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import gc
import os


if __name__ == '__main__':
    #pd.set_option('max_columns', None)
    #pd.set_option('max_rows', None)

    seed = 2022
    df_feature = pd.read_pickle('data/feature.pkl')

    for f in df_feature.select_dtypes('object').columns:
        lbl = LabelEncoder()
        df_feature[f] = lbl.fit_transform(df_feature[f].astype(str))

    df_train = df_feature[df_feature.isDefault.notna()].copy()
    df_test = df_feature[df_feature.isDefault.isna()].copy()
    print(df_train.shape, df_test.shape)
    ycol = 'isDefault'
    feature_names = list(
        filter(lambda x: x not in [ycol, 'id'], df_train.columns))
    # pycharm
    model = lgb.LGBMClassifier(objective='binary',
                               boosting_type='gbdt',
                               num_leaves=32,
                               max_depth=6,
                               learning_rate=0.05,
                               n_estimators=10000,
                               subsample=0.8,
                               feature_fraction=0.6,
                               reg_alpha=10,
                               reg_lambda=12,
                               random_state=seed,
                               is_unbalance=True,
                               metric='auc')

    df_oof = df_train[['id', ycol]].copy()
    df_oof['prob'] = 0
    prediction = df_test[['id']]
    prediction['prob'] = 0
    df_importance_list = []

    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
    for fold_id, (trn_idx, val_idx) in enumerate(
            kfold.split(df_train[feature_names], df_train[ycol])):
        X_train = df_train.iloc[trn_idx][feature_names]
        Y_train = df_train.iloc[trn_idx][ycol]

        X_val = df_train.iloc[val_idx][feature_names]
        Y_val = df_train.iloc[val_idx][ycol]

        print('\nFold_{} Training ================================\n'.format(
            fold_id + 1))

        lgb_model = model.fit(X_train,
                              Y_train,
                              eval_names=['train', 'valid'],
                              eval_set=[(X_train, Y_train), (X_val, Y_val)],
                              verbose=100,
                              early_stopping_rounds=50)

        pred_val = lgb_model.predict_proba(
            X_val, num_iteration=lgb_model.best_iteration_)[:, 1]
        df_oof.loc[val_idx, 'prob'] = pred_val

        pred_test = lgb_model.predict_proba(
            df_test[feature_names], num_iteration=lgb_model.best_iteration_)[:, 1]
        prediction['prob'] += pred_test / kfold.n_splits

        df_importance = pd.DataFrame({
            'column': feature_names,
            'importance': lgb_model.feature_importances_,
        })
        df_importance_list.append(df_importance)

        del lgb_model, pred_val, pred_test, X_train, Y_train, X_val, Y_val
        gc.collect()

    df_importance = pd.concat(df_importance_list)
    df_importance = df_importance.groupby(['column'])['importance'].agg('mean').sort_values(ascending=False).reset_index()
    auc = roc_auc_score(df_oof[ycol], df_oof['prob'])
    print('auc:', auc)
    os.makedirs('sub', exist_ok=True)

    sub = prediction[['id', 'prob']].copy()
    sub.columns = ['id', 'isDefault']
    sub.to_csv('sub/lgb_test_result_{}.csv'.format(auc), index=False)
    os.makedirs('prob', exist_ok=True)
    prediction.to_csv('prob/sub_lgb.csv', index=False)
    df_oof[['id', 'prob', ycol]].to_csv('prob/oof_lgb.csv', index=False)
    # 做了缺失值处理: auc: 0.7421851952531208
    # 不做缺失值处理: auc: 0.7427277037796024

(800000, 341) (200000, 341)

Fold_1 Training ================================

[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6
[100]	train's auc: 0.739558	valid's auc: 0.733217
[200]	train's auc: 0.747537	valid's auc: 0.73755
[300]	train's auc: 0.752945	valid's auc: 0.739387
[400]	train's auc: 0.757407	valid's auc: 0.740425
[500]	train's auc: 0.761388	valid's auc: 0.740865
[600]	train's auc: 0.765017	valid's auc: 0.741124
[700]	train's auc: 0.768447	valid's auc: 0.741419
[800]	train's auc: 0.771822	valid's auc: 0.741543
[900]	train's auc: 0.775174	valid's auc: 0.741648
[1000]	train's auc: 0.778358	valid's auc: 0.741736
[1100]	train's auc: 0.781423	valid's auc: 0.741745

Fold_2 Training ================================

[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6
[100]	train's auc: 0.739935	valid's auc: 0.732507
[200]	train's auc: 0.

In [38]:
import gc
import os
import pandas as pd
import catboost as cbt
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold


if __name__ == '__main__':
    #pd.set_option('max_columns', None)
    #pd.set_option('max_rows', None)

    seed = 2022
    df_feature = pd.read_pickle('data/feature.pkl')
    for f in df_feature.select_dtypes('object').columns:
        lbl = LabelEncoder()
        df_feature[f] = lbl.fit_transform(df_feature[f].astype(str))

    df_train = df_feature[df_feature.isDefault.notna()].copy()
    df_test = df_feature[df_feature.isDefault.isna()].copy()
    print(df_train.shape, df_test.shape)

    ycol = 'isDefault'
    feature_names = list(
        filter(lambda x: x not in [ycol, 'id'], df_train.columns))

    model = cbt.CatBoostClassifier(eval_metric='AUC',
                                   max_depth=6,
                                   learning_rate=0.05,
                                   n_estimators=10000,
                                   reg_lambda=12,
                                   # task_type='GPU',
                                   random_state=seed)

    df_oof = df_train[['id', ycol]].copy()
    df_oof['prob'] = 0
    prediction = df_test[['id']]
    prediction['prob'] = 0
    df_importance_list = []

    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
    for fold_id, (trn_idx, val_idx) in enumerate(
            kfold.split(df_train[feature_names], df_train[ycol])):
        X_train = df_train.iloc[trn_idx][feature_names]
        Y_train = df_train.iloc[trn_idx][ycol]

        X_val = df_train.iloc[val_idx][feature_names]
        Y_val = df_train.iloc[val_idx][ycol]

        print('\nFold_{} Training ================================\n'.format(
            fold_id + 1))

        cbt_model = model.fit(X_train,
                              Y_train,
                              eval_set=[(X_val, Y_val)],
                              verbose=100,
                              early_stopping_rounds=50)

        pred_val = cbt_model.predict_proba(
            X_val)[:, 1]
        df_oof.loc[val_idx, 'prob'] = pred_val

        pred_test = cbt_model.predict_proba(
            df_test[feature_names])[:, 1]
        prediction['prob'] += pred_test / kfold.n_splits

        df_importance = pd.DataFrame({
            'column': feature_names,
            'importance': cbt_model.feature_importances_,
        })
        df_importance_list.append(df_importance)

        del cbt_model, pred_val, pred_test, X_train, Y_train, X_val, Y_val
        gc.collect()

    df_importance = pd.concat(df_importance_list)
    df_importance = df_importance.groupby([
        'column'
    ])['importance'].agg('mean').sort_values(ascending=False).reset_index()
    print(df_importance)

    auc = roc_auc_score(df_oof[ycol], df_oof['prob'])
    print('auc:', auc)

    os.makedirs('sub', exist_ok=True)

    sub = prediction[['id', 'prob']].copy()
    sub.columns = ['id', 'isDefault']
    sub.to_csv('sub/cat_test_result_{}.csv'.format(auc), index=False)
    os.makedirs('prob', exist_ok=True)

    prediction.to_csv('prob/sub_cbt.csv', index=False)
    df_oof[['id', 'prob', ycol]].to_csv('prob/oof_cbt.csv', index=False)
    # 做了缺失值处理: AUC: 0.7428727778414201
    # 不做缺失值处理: AUC: 0.7433677897801529

(800000, 341) (200000, 341)

Fold_1 Training ================================

0:	test: 0.6971665	best: 0.6971665 (0)	total: 223ms	remaining: 37m 5s
100:	test: 0.7296939	best: 0.7296939 (100)	total: 10.1s	remaining: 16m 26s
200:	test: 0.7344384	best: 0.7344384 (200)	total: 19.2s	remaining: 15m 35s
300:	test: 0.7368384	best: 0.7368384 (300)	total: 28.2s	remaining: 15m 9s
400:	test: 0.7384861	best: 0.7384861 (400)	total: 37s	remaining: 14m 46s
500:	test: 0.7394207	best: 0.7394207 (500)	total: 45.7s	remaining: 14m 26s
600:	test: 0.7400946	best: 0.7400946 (600)	total: 54.2s	remaining: 14m 8s
700:	test: 0.7405985	best: 0.7405985 (700)	total: 1m 2s	remaining: 13m 51s
800:	test: 0.7408938	best: 0.7409046 (797)	total: 1m 11s	remaining: 13m 39s
900:	test: 0.7411860	best: 0.7411860 (900)	total: 1m 20s	remaining: 13m 31s
1000:	test: 0.7414844	best: 0.7414875 (997)	total: 1m 28s	remaining: 13m 18s
1100:	test: 0.7416960	best: 0.7416960 (1100)	total: 1m 37s	remaining: 13m 5s
1200:	test: 0.7418350	be

In [39]:
import pandas as pd
from sklearn.metrics import roc_auc_score


if __name__ == '__main__':
    # 官方给的测试集的概率预测
    lgb_sub = pd.read_csv('prob/sub_lgb.csv')
    xgb_sub = pd.read_csv('prob/sub_xgb.csv')
    cbt_sub = pd.read_csv('prob/sub_cbt.csv')
#     print(lgb_sub.shape)  # 200000, 2   # (200000, 2)

    lgb_oof = pd.read_csv('prob/oof_lgb.csv')
    xgb_oof = pd.read_csv('prob/oof_xgb.csv')
    cbt_oof = pd.read_csv('prob/oof_cbt.csv')
#     print(lgb_oof.shape)   # 800000, 3     id prob label
        
    
    # 三个模型测试集的预测结果合并成一个表
    sub = lgb_sub.copy()
    sub.rename({'prob': 'prob_lgb'}, axis=1, inplace=True)
    sub['prob_xgb'] = xgb_sub['prob'].values.tolist()
    sub['prob_cbt'] = cbt_sub['prob'].values.tolist()
    print(sub.shape)   # (200000, 4)   # id, prob_lgb, prob_xgb, prob_cbt
    
    # 三个模型训练集的预测结果合并成一个表
    oof = lgb_oof.copy()
    oof.rename({'prob': 'prob_lgb'}, axis=1, inplace=True)
    oof['prob_xgb'] = xgb_oof['prob'].values.tolist()
    oof['prob_cbt'] = cbt_oof['prob'].values.tolist()
    # print(oof.head())
    print(oof.shape)  # id, isDefault, prob_lgb, prob_xgb, prob_cbt

    # 计算auc
    auc1 = roc_auc_score(oof['isDefault'], oof['prob_lgb'])
    auc2 = roc_auc_score(oof['isDefault'], oof['prob_xgb'])
    auc3 = roc_auc_score(oof['isDefault'], oof['prob_cbt'])
    print('lgb的auc:', auc1)   # 0.7427987878983419
    print('xgb的auc:', auc2)   # 0.7434926337868603
    print('cbt的auc:', auc3)   # 0.4994859248920798
    # lgb的auc: 0.7427277037796024
    # xgb的auc: 0.7434986214509369
    # cbt的auc: 0.7433677897801529
    
    oof['three'] = (oof['prob_lgb'] + oof['prob_xgb'] + oof['prob_cbt'])/3
    avg_auc = roc_auc_score(oof['isDefault'], oof['three'])
    print('三个均值的auc:', avg_auc)   
        
    
    w_lgb = 0.5
    w_xgb = 0.5
    w_cbt = 0.5
    # x y x = 根号x * 根号y * 根号 z
    oof['prob'] = oof['prob_lgb'] ** w_lgb * oof['prob_xgb'] ** w_xgb * oof['prob_cbt'] ** w_cbt
    auc = roc_auc_score(oof['isDefault'], oof['prob'])
    print('根号相乘的auc:', auc)
    
    oof['both'] = (oof['prob_xgb'] + oof['prob_cbt'])/2
    
    both_auc = roc_auc_score(oof['isDefault'], oof['both'])
    print('xgb and cbt:', both_auc)
    
    
    sub['prob'] = sub['prob_lgb'] ** w_lgb * sub['prob_xgb'] ** w_xgb * sub['prob_cbt'] ** w_cbt
    # sub['prob'] = (sub['prob_lgb'] + sub['prob_xgb']) / 2
    sub.rename({'prob': 'isDefault'}, axis=1, inplace=True)
    sub[['id', 'isDefault']].to_csv('sub/20221012.csv', index=False)


(200000, 4)
(800000, 5)
lgb的auc: 0.7421851952531208
xgb的auc: 0.7430355546194591
cbt的auc: 0.7428531664426238
三个均值的auc: 0.7436647734570626
根号相乘的auc: 0.7438512708269246
xgb and cbt: 0.743844773578163


# 面试题
## 1. xgboost为什么要使用泰勒展开？
如果不使用泰勒展开，在进行公式推导时，损失函数就必须是确定的。为了统一进行推导，我们进行了泰勒的二阶展开。

## 2. xgboost在优化的时候，采用了哪些近似？
1. 从特征数的角度去考虑: 使用的列采样（包含按树随机、按层随机）。按树随机是在构建当前树的时候，随机抽若干特征，在后面逐个节点构造的时候，都采用的是这些特征。 按层随机是在构建当前树的时候，每层的节点都需要进行特征的随机抽取。
2. 从特征取值的角度去考虑(主要是找切分点)，有两种方案: 
    - 贪心算法: 对特征进行预排序，然后分别以每一个取值当做一个切分点，然后计算增益。好处是能找到全局最优的分割点，坏处是当特征的取值非常多的时候，速度太慢了。
    - 近似算法: 先对特征进行预排序，然后找出其分位点(这个分位点一般是二分位点、三分为点等等)， 我们主要计算每个分位点作为分割点的增益情况。优点是算的快，缺点是可能不是全局最优点。
    
## 3. xgboost中缺失值的处理？
假设有10个样本，对于特征x的取值都是nan,如果按正常思维，我们可以分别将每一个样本分到左子树或者右子树，看起哪种增益下降的多，就放到哪个子树上，但显然这是理想化的。当确实样本非常多的时候，效率是非常低下的。在xgboost中，是将当前所有取值为nan的样本分别放到左子树或者右子树，看哪种损失下降多，直接将所有缺失值样本放到某棵子树上。

## 4. xgboost是怎样防止过拟合的？
第一点就是构造了正则项:$\Omega(f_t) = \gamma T + \frac{1}{2}\lambda \sum_{j=1}^Tw_j^2$ 让叶子节点变少或者权重变小，防止一棵树过度拟合。

第二点使用了收缩率(shrinkage)机制。即: 在xgboost中加入了步长$\eta$,具体公式:
$$\hat y_i^t = \hat y_i^{t-1} + \eta f_t(x_i)$$
也就是每棵树的预测结果，最后都得收缩一下，避免过拟合。

## 5. lightgbm？
LightGBM = Xgboost + Histogram + GOSS + EFB
- Histogram算法: 直方图算法
- GOSS算法: 基于梯度的单边采样算法（Gradient-based One-Side Sampling）
- EFB算法: 互斥特征捆绑算法 (Exclusive Feature Bunding)

### (1). Histogram算法:

   XGBoost中默认的算法对于决策树的学习使用基于 pre-sorted 的算法，这是一个简单的解决方案，但是不易于优化。

   预排序算法的优点是能精确地找到分割点。但是缺点也很明显：首先，空间消耗大。这样的算法需要保存数据的特征值，还保存了特征排序的结果（例如，为了后续快速的计算分割点，保存了排序后的索引），这就需要消耗训练数据两倍的内存。其次，时间上也有较大的开销，在遍历每一个分割点的时候，都需要进行分裂增益的计算，消耗的代价大。
    
   LightGBM 利用基于histogram的算法，通过将连续特征（属性）值分段为 discrete bins 来加快训练的速度并减少内存的使用。
   直方图其实可以分为频率直方图和频数直方图，横轴为相关数据，纵坐标为该数据出现的频率或者频数。画出如下直方图:
    <div>
    <img src="./asset/hist.png" width="400"/>
    </div>
    直方图算法，简单来说，就是对特征值进行装箱处理，把连续的浮点特征值离散化成k个整数，形成一个一个箱体(bins),同时构造一个宽度为k的直方图，在遍历数据的时候，根据离散化后的值作为索引在直方图中累计统计量（因此这里是频数直方图），当遍历一次数据后，直方图累积了需要的统计量，然后根据直方图的离散值，遍历寻找最优的分割点。
    
   对于连续特征来说，装箱处理就是特征工程中的离散化: 如[0,10)区间的值都可以赋值为0，[10,20)区间的值都可以赋值为1等。这样就可以把众多的数值换分到有限的分箱中，在LightGBM中默认的分箱数(bins)为256.
    
   eg: 假设特征x取值范围为0.1-0.9, 假定分为3个箱子（bin），此时宽度就是3。并且这样离散：[0.1,0.3)->0, [0.3,0.6)->1, [0.6,0.9)->2. 

### (2). 决策树生长策略:

   对于大部分决策树算法使用的时level-wise生成策略，即同一层的叶子节点每次都一起分裂，但实际上一些叶子节点的分类增益较低，这样会增加不小的开销，如下图所示:
    <div>
    <img src="./asset/level-wise.png" width="400"/>
    </div>
    
   LightGBM使用的leaf-wise生长策略，每次在当前叶子节点中，找出分类增益最大的叶子节点进行分裂，而不是所有叶子节点都进行分裂，这样会提高精度。如下图所示:
    <div>
    <img src="./asset/leaf-wise.png" width="400"/>
    </div>
    
### (3). GOSS算法:
通过对样本采样的方法来减少计算目标函数增益时候的复杂度。GOSS算法中，梯度更大的样本点在计算信息增益时更占更大的比重，当我们对样本进行下采样的时候保留这些梯度较大的样本，并随即剔除梯度小的样本。
> lightgbm做法: 首先把样本按照梯度排序，选出梯度最大的a%的样本，然后再剩余梯度下的样本中随机选取b%个样本，在计算信息增益的时候，将选出来的b%小梯度样本的信息增益扩大（1-a）/b的倍数。
    
### (4). EFB算法:
- EFB算法将互斥特征绑在一起以减少特征维度。
- EFB算法可以有效减少用于构建直方图的特征数量，从而降低计算复杂度，尤其是特征中包含大量稀疏特征的时候。
> lightGBM可以直接将每个类别取值和一个bin关联，从而自动的处理它们，而无需预处理成onehot编码。


## 6. 为什么需要将离散特征转为onehot的形式，和其他形式比较，onehot的好处是？
假设有一个特征为性别，特征的取值分别为:[男、女、男、男、中性、女]   
采用one-hot编码: 男 女 中性 [x, x, x]

[1, 0, 0]

[0, 1, 0]

[1, 0, 0]

[1, 0, 0]

[0, 0, 1]

[0, 1, 0]

如果采用labelencode:
[0, 1, 0, 0, 2, 1]

将离散特征转为one-hot编码，可以让特征之间的距离计算更佳合理。

## 8. CatBoost？
与XGBoost、LightGBM相比，CatBoost的创新点有：
- 嵌入了自动将类别型特征处理为数值型特征的创新算法。首先对categorical features做一些统计，计算某个类别特征（category）出现的频率，之后加上超参数，生成新的数值型特征（numerical features）。
- Catboost还使用了组合类别特征，可以利用到特征之间的联系，这极大的丰富了特征维度。
- 采用排序提升的方法对抗训练集中的噪声点，从而避免梯度估计的偏差，进而解决预测偏移的问题。
- 采用了完全对称树作为基模型。

### (1). CatBoost中的TS(target statistics)编码

对于一些类别基数比较小的特征，例如2~3类，可以直接使用one-hot编码，但对于类别基数较大的特征，one-hot编码会产生特征维度问题，这种情况下TS编码会合适一些。 TS编码是一种有监督编码，其做法是：将类别取值转换为该类别为正类的概率。用公式来说明:
<div>
<img src="attachment:image.png" width="400"/>
</div>


<div>
<img src="attachment:image-10.png" width="400"/>
</div>

### (2). 预测偏移处理

   在GBDT算法中，每一棵树都是为了拟合前一棵树上的梯度，构造树时所有的样本都参与了，一个样本参与了建树，然后又用这棵树去估计样本值，这样的估计就不是无偏估计，当测试集和训练集上的样本分布不一致时，模型就会因过拟合而性能不佳，即在测试集上产生了预测偏移。

   对于样本$x_i$,如果用一个不包含它的模型去估计它的梯度，估计结果可以视为无偏估计。基于这种思路，CatBoost算法中采用了如下策略：在每一轮迭代时，将样本集排序，然后训练n个模型$M_i，i=1,...,n$为样本数量，其中$M_i$是由前i个样本训练得到(基于本轮样本的排序，包含样本i)，然后估计样本的梯度时，使用模型$M_{i-1}$来估计，因为$M_{i-1}$是由不包含样本i的样本训练得到，因此该估计结果是无偏估计。
   <div>
    <img src="attachment:image-2.png" width="400"/>
    </div>
    
   这种方式尽管得到了无偏估计，但是对于排序靠前的样本，它的梯度估计结果可能并不太准确，具有较大的方差，因为估计它的模型是由较少的样本训练的，而且是基于本轮迭代中的样本排序计算的，为了减少预测方差，CatBoost在每轮迭代中都会对样本进行重新排序，然后按照相同的思路估计本轮中样本的梯度，这样多轮迭代的最终结果就可以获取一个较小的方差。
    
### (3). 对称树
首先看一下普通的决策树:
    <div>
    <img src="attachment:image-3.png" width="400"/>
    </div>
上面这棵树中，第二层中左右两个节点的判断条件不同。如果一个样本进入了左面的节点，下一个问题是是否关注了“机器会学习”，如果进入了右面的节点，下一个问题则是“喜欢tensorflow”还是“pytorch”。

那么对称树呢？和普通决策树有什么区别？
    <div>
    <img src="attachment:image-4.png" width="400"/>
    </div>
对称树中，每一层的每一个节点判断条件都是一样的。假如如果我们只训练一棵树，那么显然对称树的overfit能力会比普通的决策树弱（因为同样高度的树，它用的特征少，也就约束少，也就不容易过拟合）；但是在GBM中，我们通常训练很多的树，所以overfit的能力不必担心。那么对称树在GBM中有什么优势呢？下面列出两点：

- 拟合模式相对简单，因为每一层都是一个判断条件
- 可以提高预测速度
    
最大的优势就是第二点——预测速度会提升。为什么会更快呢？   
    <div>
    <img src="attachment:image-5.png" width="400"/>
    </div>
    <div>
    <img src="attachment:image-6.png" width="400"/>
    </div>

上面这棵树最底层有四个节点，我们可以把这四个节点从0到3编号（index），每一个节点有一个对应值（value）。比如一个样本最终进入了第一个节点，那么这棵树对这个样本的预测值就是1.6。我们可以把value存放在一个数组里面，这样给定index就可以立刻得到value。那么对一个样本用一棵树进行预测时，我们只需要找到这个样本对应的index，因为在对称树中每一层的判断条件都是一样的，所以每一层都可以用0或者1来表示，比如是学生，用1表示，不是用0；有关注机器会学习用1表示，没有用0，那么每一个样本进入这棵树，都可以用两个bits来表示，也就是四种可能结果：00，01，10，和11。这两个bits代表的数字就是index。这意味着什么呢？我们在做预测的时候，只要对每一层的条件进行判断，然后就可以找到index。因为每一个树的结构（判断条件）是已知的，我们甚至可以对这个过程进行并行计算。

如果有个特征是小学几年级，那这个树可能就是六叉树。

目前对称树在catboost中已经实现，xgboost和lightgbm都依然使用传统的决策树结构。传统的决策树对每一个样本进行预测时都要从根节点遍历这棵树，但是应用catboost中的对称树我们只要进行数组的操作。下面的模型预测速度对比结果来自catboost的官方网站[3]（40万样本，2000个特征），可见对称树在预测速度上的优异表现。
    

In [3]:
df_test.head()

,id,loanAmnt,term,interestRate,installment,grade,subGrade,employmentTitle,employmentLength,homeOwnership,...,title_subGrade_default_ratio,title_employmentTitle_default_ratio,title_employmentLength_default_ratio,title_homeOwnership_default_ratio,title_verificationStatus_default_ratio,title_purpose_default_ratio,title_postCode_default_ratio,title_regionCode_default_ratio,title_initialListStatus_default_ratio,title_applicationType_default_ratio
800000,800000,14000.0,3,10.99,458.28,2,12,7027.0,10,0,...,0.133012,0.045455,0.213966,0.187219,0.159736,0.217988,0.253369,0.229959,0.215726,0.216735
800001,800001,20000.0,5,14.65,472.14,3,19,60426.0,10,0,...,0.254797,NaN,0.180434,0.169427,0.134107,0.181606,0.205521,0.158195,0.175684,0.181111
800002,800002,12000.0,3,19.99,445.91,4,23,23547.0,2,1,...,0.338314,0.000000,0.219860,0.254131,0.263673,0.217988,0.325203,0.266843,0.222059,0.216735
800003,800003,17500.0,5,14.31,410.02,3,18,636.0,4,0,...,0.278401,0.000000,0.162484,0.148977,0.182951,0.175241,0.195946,0.187879,0.174241,0.175009
800004,800004,35000.0,3,17.09,1249.42,4,20,368446.0,0,1,...,0.290705,NaN,0.232057,0.254131,0.223867,0.217988,0.211215,0.215245,0.215726,0.216735
